In [1]:
!nvidia-smi

Fri Dec 22 17:19:14 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:81:00.0 Off |                  Off |
| 30%   27C    P8              25W / 450W |      3MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from IPython.display import display, Image
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.229 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
Setup complete ✅ (32 CPUs, 251.7 GB RAM, 28.9/332.3 GB disk)


### Ultralytics Settings

In [ ]:
from ultralytics import settings

# View all settings
print(settings)

# Return a specific setting
value = settings['runs_dir']

#### Modifying Settings
In this code chunk I would like to modify the saving runs output dir to be inside the thesis repo

In [ ]:
from ultralytics import settings

models_output_path = ""
# Update a setting
settings.update({'runs_dir': models_output_path})

# Update multiple settings
settings.update({'runs_dir': '/path/to/runs', 'tensorboard': False})

# Reset settings to default values
# settings.reset()

### Prepare Dataset
In this section, the dataset along with YOLO annotations is processed to conform to the input structure required by YOLO. The YOLO format necessitates organizing the dataset into two main directories, `images` and `labels`. Each of these directories is further divided into two subdirectories, `train` and `val`, for training and validation datasets respectively. This structure facilitates YOLO's training and validation processes.

In [ ]:
from src.segmentation.framework_handlers.yolov8_handler import split_data
# Parameters
image_dir = '/home/etaylor/code_projects/thesis/segments/etaylor_all_cannabis_patches_multi_class/v0.1'
label_dir = '/home/etaylor/code_projects/thesis/segments/etaylor_all_cannabis_patches_multi_class/annotations/yolo/labels/export_coco-instance_etaylor_all_cannabis_patches_multi_class_v0.1'
train_percentage = 0.8  # 80% of data for training
organized_dataset_path = '/home/etaylor/code_projects/thesis/segments/etaylor_all_cannabis_patches_multi_class/annotations/yolo/organized_dataset'

# Run the function
split_data(image_dir, label_dir, train_percentage, organized_dataset_path)


### Create the YAML Config file
The YAML file in the context of fine-tuning a YOLO model serves as a configuration guide, outlining the dataset structure and specifying class names for the training and validation of the model.

In [3]:
yaml_path = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/instance_segmentation/data.yaml"

In [ ]:
from src.segmentation.framework_handlers.yolov8_handler import create_yaml
create_yaml(organized_dataset_path, yaml_path)


### Find best Hyper params with [Ray Tune](https://docs.ultralytics.com/integrations/ray-tune/#custom-search-space-example)

In the provided code snippet, Ray Tune is utilized for hyperparameter tuning of a YOLOv8n segmentation model using a specific model checkpoint and dataset defined in a YAML file.

In [4]:
from ultralytics import YOLO

model_checkpoint_path = '/home/etaylor/code_projects/thesis/checkpoints/yolo/yolov8n-seg.pt'
model = YOLO(model_checkpoint_path) # pass any model type

# Start tuning hyperparameters for YOLOv8n training on the COCO8 dataset
result_grid = model.tune(data=yaml_path, use_ray=True)

(_tune pid=25200) Ultralytics YOLOv8.0.229 🚀 Python-3.10.13 torch-2.1.1+cu121 CPU (AMD EPYC 7302 16-Core Processor)
(_tune pid=25200) engine/trainer: task=segment, mode=train, model=/home/etaylor/code_projects/thesis/checkpoints/yolo/yolov8n-seg.pt, data=/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/instance_segmentation/data.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes

train: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_all_cannabis_patches_multi_class/annotations/yolo/organized_dataset/labels/train.cache... 43 images, 37 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<?, ?it/s]
val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_all_cannabis_patches_multi_class/annotations/yolo/organized_dataset/labels/val.cache... 7 images, 13 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


(_tune pid=25200) Plotting labels to /home/etaylor/runs/segment/train5/labels.jpg... 


  0%|          | 0/5 [00:00<?, ?it/s]
train: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_all_cannabis_patches_multi_class/annotations/yolo/organized_dataset/labels/train.cache... 43 images, 37 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<?, ?it/s] [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_all_cannabis_patches_multi_class/annotations/yolo/organized_dataset/labels/val.cache... 7 images, 13 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s] [repeated 3x across cluster]


(_tune pid=25195) optimizer: 'optimizer=auto' found, ignoring 'lr0=0.005715112775406568' and 'momentum=0.7004323030795154' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
(_tune pid=25195) optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0008364252222891914), 76 bias(decay=0.0)
(_tune pid=25195) 100 epochs...
(_tune pid=25195)       Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size
(_tune pid=25198) Ultralytics YOLOv8.0.229 🚀 Python-3.10.13 torch-2.1.1+cu121 CPU (AMD EPYC 7302 16-Core Processor) [repeated 3x across cluster]
(_tune pid=25198) engine/trainer: task=segment, mode=train, model=/home/etaylor/code_projects/thesis/checkpoints/yolo/yolov8n-seg.pt, data=/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/instance_segmentation/data.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, worker

...

  0%|          | 0/5 [00:00<?, ?it/s]


(_tune pid=25200) 
(_tune pid=25200)       Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size
(_tune pid=1057)                    all         20        129      0.561      0.645      0.605      0.393      0.558      0.641      0.607      0.376


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:10<00:00, 10.78s/it]


(_tune pid=1057) 
(_tune pid=1057)       Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/5 [00:00<?, ?it/s]
     83/100         0G    0.02181    0.03604      1.159     0.9477        270        640:  40%|████      | 2/5 [00:39<00:58, 19.49s/it]
     68/100         0G    0.02286    0.03849      2.959      1.034        371        640:  20%|██        | 1/5 [00:20<01:21, 20.32s/it]
     77/100         0G   0.008683    0.01524      11.72      1.088        322        640:  60%|██████    | 3/5 [00:58<00:39, 19.58s/it]
     84/100         0G    0.01131    0.01898      9.255      1.043        269        640:  20%|██        | 1/5 [00:20<01:22, 20.62s/it]
     83/100         0G    0.02091    0.03494       1.13      0.958        446        640:  60%|██████    | 3/5 [01:02<00:42, 21.48s/it]


### Basic Trial-Level Analysis
Access individual trial hyperparameter configurations and the last reported metrics.



In [ ]:
for i, result in enumerate(result_grid):
    print(f"Trial #{i}: Configuration: {result.config}, Last Reported Metrics: {result.metrics}")

#### Plotting the Entire History of Reported Metrics for a Trial


In [ ]:
import matplotlib.pyplot as plt

for result in result_grid:
    plt.plot(result.metrics_dataframe["training_iteration"], result.metrics_dataframe["mean_accuracy"], label=f"Trial {i}")

plt.xlabel('Training Iterations')
plt.ylabel('Mean Accuracy')
plt.legend()
plt.show()

### Fine tune YOLOv8 Model on my dataset

In [ ]:
model_checkpoint_path = '/home/etaylor/code_projects/thesis/checkpoints/yolo/yolov8n-seg.pt'
model = YOLO(model_checkpoint_path)

# regular train of the model
results = model.train(data=yaml_path, epochs=250, imgsz=512)

In [ ]:
valid_results = model.val()
print(valid_results)

In [ ]:
from IPython.display import Image as show_image
show_image(filename="/sise/home/etaylor/runs/segment/train22/val_batch0_pred.jpg")